<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_ELECTRICITY_Step4-VAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import acf
from sklearn.ensemble import RandomForestRegressor
from numpy import arange
#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

#-----------for whole dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data_V3.csv')


#-----------for test dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_window_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/derived_window_label_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/test_data_V3.csv')

n_features = 5
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))

sample_size = 100
#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------



plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)

# Import libraries
import matplotlib.pyplot as plt
import numpy as np
fig = plt.figure(figsize =(10, 7))
# Creating plot
plt.boxplot(y_pred)
# show plot
plt.show()

fx_window1 = 4
fx_window2 = 7
fx_window3 = 9
fx_window4 = 11


pred_step =1

############################METHOD 1---------------- GO BY AIC ---------------------------------------------------------------------------------

aic1_count = list()
aic2_count = list()
aic3_count = list()
aic4_count = list()
aic5_count = list()
aic6_count = list()





for i in range(sample_size) :

  x1 = x[i]
  x_train=x1[:-pred_step]
  x_test=x1[-pred_step:]
  regressor= VAR(x_train)
  results1 = regressor.fit(fx_window1)
  results2 = regressor.fit(fx_window2)
  results3 = regressor.fit(fx_window3)
  results4 = regressor.fit(fx_window4)
  #results5 = regressor.fit(fx_window5)
  results6 = regressor.fit(int(y_pred[i]))

#Calculate AIC for ach case--------------
  try:
    aic1 = results1.aic
  except:
    aic1 = 9999
  aic1_count.append(aic1)

  try:
    aic2 = results2.aic
  except:
    aic2 = 9999
  aic2_count.append(aic2)

  try:
    aic3 = results3.aic
  except:
    aic3 = 9999
  aic3_count.append(aic3)

  try:
    aic4 = results4.aic
  except:
    aic4 = 9999
  aic4_count.append(aic4)

  #try:
  #  aic5 = results5.aic
  #except:
  #  aic5 = 9999
  #aic5_count.append(aic5)


  try:
    aic6 = results6.aic
  except:
    aic6 = 9999
  aic6_count.append(aic6)


print("aic1_count : ", aic1_count)
print("aic2_count : ", aic2_count)
print("aic3_count : ", aic3_count)
print("aic4_count : ", aic4_count)
#print("aic5_count : ", aic5_count)
print("aic6_count : ", aic6_count)

df = pd.DataFrame({'AIC1' : aic1_count, 'AIC2' : aic2_count, 'AIC3' : aic3_count,  'AIC4':aic4_count, 'AIC6 - Predicted' : aic6_count})
df[0:1000].plot(figsize=(15,6))
plt.legend()
plt.show()


df.to_csv('/content/drive/MyDrive/PHD/2024/PREDICTION_COMPARISON/Electricity_AIC_comparison.csv', index = False)


################---------------------------METHOD 2- -GO BY RMSE----------------------------------------------------####################
rmse_V1 = list()
rmse_V2 = list()
rmse_V3 = list()
rmse_V4 = list()
rmse_V5 = list()
rmse_V6 = list()

for i in range(100) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  rmse_V1_1 =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
  rmse_V1.append(rmse_V1_1)
  print(rmse_V1_1)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  rmse_V1_2 =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
  rmse_V2.append(rmse_V1_2)
  print(rmse_V1_2)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  rmse_V1_3 =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
  rmse_V3.append(rmse_V1_3)
  print(rmse_V1_3)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  rmse_V1_x =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
  rmse_V4.append(rmse_V1_x)
  print(rmse_V1_x)

print(np.mean(rmse_V1), np.mean(rmse_V2),  np.mean(rmse_V3), np.mean(rmse_V4))




df.to_csv('/content/drive/MyDrive/PHD/2024/PREDICTION_COMPARISON/Electricity_mse_comparison.csv', index = False)


0.008901944382957761
0.008901944382957761
0.008901944382957761
0.008901944382957761
0.00013073473588468156
0.00013073473588468156
0.00013073473588468156
0.00013073473588468156
0.003725935493742829
0.003725935493742829
0.003725935493742829
0.003725935493742829
0.0012642379914154748
0.0012642379914154748
0.0012642379914154748
0.0012642379914154748
0.2509393546472456
0.2509393546472456
0.2509393546472456
0.2509393546472456
0.01439065818516775
0.01439065818516775
0.01439065818516775
0.01439065818516775
0.0010622331185394184
0.0010622331185394184
0.0010622331185394184
0.0010622331185394184
0.012949231461814406
0.012949231461814406
0.012949231461814406
0.012949231461814406
0.004217757343214367
0.004217757343214367
0.004217757343214367
0.004217757343214367
0.002137462683357628
0.002137462683357628
0.002137462683357628
0.002137462683357628
0.03953403811381492
0.03953403811381492
0.03953403811381492
0.03953403811381492
0.26128497775852594
0.26128497775852594
0.26128497775852594
0.26128497775852

In [ ]:
from google.colab import drive
drive.mount('/content/drive')